### 감정 분석

In [1]:
import time
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.output_parsers.structured import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.exceptions import OutputParserException

import apitool



llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) # Key는 환경 변수 자동 인식, OPENAI_API_KEY

In [2]:
# 1) 데이터 준비
article = '''
    속도도 빠르고 가격도 저렴하며 소리도 조용한 노트북입니다. 추천합니다.
'''
article = apitool.remove_empty_lines(article) # 빈라인 삭제

# 2) 출력 스키마 & 파서 설정
response_schemas = [
    ResponseSchema(name="res", description="긍정: 1, 부정: 0")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# 3) PromptTemplate 정의
prompt = PromptTemplate.from_template(
    "{system}\n"
    "아래 댓글이 부정적인지 긍정적인지 알려줘('res': 1 또는 0).\n\n{article}\n\n"
    "{format_instructions}"
)

inputs = {
    "system": "너는 댓글 긍정, 부정 판단 시스템이야",
    "article": article,
    "format_instructions": format_instructions
}

# (디버깅용) 실제 프롬프트 문자열 확인
# rendered = prompt.format(**inputs)
# print("-> prompt:", rendered)

# 4) 파이프라인 실행 & 결과 출력 (Prompt → LLM → Parser)
pipeline = prompt | llm | output_parser

result = pipeline.invoke(inputs)
print("-> response: ", type(result))
print("-> response: ", result)

-> response:  <class 'dict'>
-> response:  {'res': 1}


### 요약 기능

In [3]:
# 1) 데이터 준비
article = '''
벌써 여름이 된 것처럼 후텁지근한 하루였습니다.
강릉에서는 한여름 더위가 나타나면서 시민들이 바닷가로 뛰어들기도 했는데요.
영덕은 35도, 서울 동작구도 30도 안팎까지 올라서 올 들어 가장 더웠습니다.
또 곳곳에서는 5월 중순 기준 가장 높은 기온을 기록했습니다.
경주가 무려 33.7도까지 치솟았는데요.
내일도 오늘과 비슷한 더위가 이어지겠고요.
특히 습도가 높아서 체감 온도는 1~2도가량 더 높겠습니다.
절기 소만이기도 한 내일은 곳곳에서 비나 소나기가 내리겠습니다.
새벽부터 오전 사이 수도권과 강원 내륙, 남해안과 제주에 비가 내리는데요.
제주 산지에는 60밀리미터 이상의 다소 많은 비가 내리겠고요.
오후부터 밤사이에는 경기 동부를 비롯한 내륙 곳곳에 소나기가 내릴 텐데 이 벼락이 치고 돌풍이 불 수 있어서 주의하셔야겠습니다.
내일은 전국의 하늘빛이 흐리겠고요.
서해안과 남해안 지방은 짙은 안개가 끼겠습니다.
내일 아침 기온은 서울이 21도로 오늘보다 조금 높겠습니다.
한낮 기온은 서울이 27도, 대구 29도까지 오르겠고요.
영동 지방은 오늘보다 8도가량 낮겠습니다.
모레는 전국에 또 비가 내리겠고 고온은 한풀 꺾일 전망입니다.
'''
article = apitool.remove_empty_lines(article) # 빈라인 삭제

# 2) 출력 스키마 & 파서 설정
response_schemas = [
    ResponseSchema(name="res", description="{'res': 요약된 문장}")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# 3) PromptTemplate 정의
prompt = PromptTemplate.from_template(
    "{system}\n"
    "아래 문장을 100자 이내로 한글로 요약해줘.\n\n{article}.\n\n"
    "{format_instructions}"
)

inputs = {
    "system": "너는 요약 시스템이야",
    "article": article,
    "format_instructions": format_instructions
}

# (디버깅용) 실제 프롬프트 문자열 확인
# rendered = prompt.format(**inputs)
# print("-> prompt:", rendered)

# 4) 파이프라인(체인) 구성 및 실행
pipeline = prompt | llm | output_parser

for attempt in range(3): # 0 ~ 2
    try:
        result = pipeline.invoke(inputs)
        break
    except (OutputParserException) as e:
        if attempt < 2:
            continue
        else:
            raise # 에러 출력
        
print("-> response:", result["res"])

-> response: 여름처럼 더운 날씨가 이어지며 강릉은 한여름 더위, 경주는 33.7도 기록. 내일 비와 소나기 예보.


### 번역 기능

In [ ]:
# 1) 데이터 준비
title = '다음주 MT 참석 부탁합니다.'
content = '''
다음주 진행되는 MT에서 많은 상품을 시상할 예정이니 꼭 참석 부탁합니다

- 상품 목록 -
1. 노트북
2. 프린터기
3. 외장하드 2TB
'''
article = '''
벌써 여름이 된 것처럼 후텁지근한 하루였습니다.
강릉에서는 한여름 더위가 나타나면서 시민들이 바닷가로 뛰어들기도 했는데요.
영덕은 35도, 서울 동작구도 30도 안팎까지 올라서 올 들어 가장 더웠습니다.
또 곳곳에서는 5월 중순 기준 가장 높은 기온을 기록했습니다.
경주가 무려 33.7도까지 치솟았는데요.
내일도 오늘과 비슷한 더위가 이어지겠고요.
특히 습도가 높아서 체감 온도는 1~2도가량 더 높겠습니다.
절기 소만이기도 한 내일은 곳곳에서 비나 소나기가 내리겠습니다.
새벽부터 오전 사이 수도권과 강원 내륙, 남해안과 제주에 비가 내리는데요.
제주 산지에는 60밀리미터 이상의 다소 많은 비가 내리겠고요.
오후부터 밤사이에는 경기 동부를 비롯한 내륙 곳곳에 소나기가 내릴 텐데 이 벼락이 치고 돌풍이 불 수 있어서 주의하셔야겠습니다.
내일은 전국의 하늘빛이 흐리겠고요.
서해안과 남해안 지방은 짙은 안개가 끼겠습니다.
내일 아침 기온은 서울이 21도로 오늘보다 조금 높겠습니다.
한낮 기온은 서울이 27도, 대구 29도까지 오르겠고요.
영동 지방은 오늘보다 8도가량 낮겠습니다.
모레는 전국에 또 비가 내리겠고 고온은 한풀 꺾일 전망입니다.
'''
content = apitool.remove_empty_lines(content) # 빈라인 삭제

# 2) 출력 스키마 & 파서 설정
response_schemas = [
    ResponseSchema(name="res", description="{'title': 번역된 문장, 'content': 번역된 문장}")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# 3) PromptTemplate 정의
prompt = PromptTemplate.from_template(
    "{system}\n"
    "아래 문장을 {language}로 번역해줘.\n\n"
    "{title}\n\n"
    "{content}\n\n"
    "{format_instructions}"
)

inputs = {
    "system": "너는 요약 시스템이야",
    "language": language,
    "title": title,
    "content": content,
    "format_instructions": format_instructions
}

# (디버깅용) 실제 프롬프트 문자열 확인
# rendered = prompt.format(**inputs)
# print("-> prompt:", rendered)

# 4) 파이프라인(체인) 구성 및 실행
pipeline = prompt | llm | output_parser

for attempt in range(3): # 0 ~ 2
    try:
        result = pipeline.invoke(inputs)
        break
    except (OutputParserException) as e:
        if attempt < 2:
            continue
        else:
            raise # 에러 출력
        
print("-> response:", result["res"])